In [363]:
import pandas as pd

In [364]:
df = pd.read_csv('data/clean_data.csv')

In [365]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44564 entries, 0 to 44563
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Delivery_person_Age             44564 non-null  float64
 1   Delivery_person_Ratings         44564 non-null  float64
 2   Weather_conditions              44564 non-null  object 
 3   Road_traffic_density            44564 non-null  object 
 4   Vehicle_condition               44564 non-null  int64  
 5   Type_of_order                   44564 non-null  object 
 6   Type_of_vehicle                 44564 non-null  object 
 7   multiple_deliveries             44564 non-null  float64
 8   Festival                        44564 non-null  object 
 9   City                            44564 non-null  object 
 10  Time_taken (min)                44564 non-null  int64  
 11  Week_days                       44564 non-null  object 
 12  Time_Orderd_Hours               

In [366]:
X = df.iloc[:,[0,1,3,7,8,16]]

In [392]:
['Time_taken (min)', 'Road_traffic_density', 'multiple_deliveries',
       'Delivery_person_Ratings', 'Distance_Resturant_to_Location',
       'Delivery_person_Age', 'Festival', 'Vehicle_condition',
       'Time_Order_picked_Hours', 'Time_Orderd_Hours', 'Type_of_vehicle',
       'City']

['Time_taken (min)',
 'Road_traffic_density',
 'multiple_deliveries',
 'Delivery_person_Ratings',
 'Distance_Resturant_to_Location',
 'Delivery_person_Age',
 'Festival',
 'Vehicle_condition',
 'Time_Order_picked_Hours',
 'Time_Orderd_Hours',
 'Type_of_vehicle',
 'City']

In [368]:
X

,Delivery_person_Age,Delivery_person_Ratings,Road_traffic_density,multiple_deliveries,Festival,Distance_Resturant_to_Location
0,36.0,4.2,Jam,3.0,No,10.280582
1,21.0,4.7,High,1.0,No,6.242319
2,23.0,4.7,Medium,1.0,No,13.787860
3,34.0,4.3,Low,0.0,No,2.930258
4,24.0,4.7,Jam,1.0,No,19.396618
...,...,...,...,...,...,...
44559,30.0,4.8,High,0.0,No,1.489846
44560,21.0,4.6,Jam,1.0,No,11.007735
44561,30.0,4.9,Low,0.0,No,4.657195
44562,20.0,4.7,High,1.0,No,6.232393


In [369]:
y = df['Time_taken (min)']

In [370]:
y

0        46
1        23
2        21
3        20
4        41
         ..
44559    32
44560    36
44561    16
44562    26
44563    36
Name: Time_taken (min), Length: 44564, dtype: int64

In [371]:
cat_cols = X.select_dtypes(include='object').columns

In [372]:
num_cols = X.select_dtypes(exclude='object').columns

In [373]:
cat_cols

Index(['Road_traffic_density', 'Festival'], dtype='object')

In [374]:
Weather_conditions_categories =['Fog', 'Stormy', 'Sandstorms', 'Windy', 'Cloudy', 'Sunny']
Road_traffic_density_categories=['Jam', 'High', 'Medium', 'Low']
Type_of_order_categories=['Snack', 'Meal', 'Drinks', 'Buffet']
Type_of_vehicle_categories=['motorcycle', 'scooter', 'electric_scooter']
Festival_categories = ['No', 'Yes']
City_categories = ['Metropolitian', 'Urban', 'Semi-Urban']
Week_days_categories = ['Saturday', 'Sunday', 'Friday', 'Monday', 'Tuesday', 'Wednesday','Thursday']

In [375]:
df['Week_days'].unique()

array(['Saturday', 'Sunday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'], dtype=object)

In [376]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [377]:
num_pipeline= Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='median')),
        ("scaler", StandardScaler())
    ]
)

In [378]:
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        # ('ordinalencoder', OrdinalEncoder(categories=[Weather_conditions_categories,Road_traffic_density_categories,Type_of_order_categories,Type_of_vehicle_categories,Festival_categories,City_categories,Week_days_categories])),
        ('ordinalencoder', OrdinalEncoder(categories=[Road_traffic_density_categories,Festival_categories])),
        ('scaler', StandardScaler())
    ]
)


In [379]:
preprocessor=ColumnTransformer([
    ('num_pipeline', num_pipeline,num_cols ),
    ('cat_pipeline', cat_pipeline, cat_cols)
])

In [380]:
from sklearn.model_selection import train_test_split

In [381]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [382]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [383]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__multiple_deliveries,num_pipeline__Distance_Resturant_to_Location,cat_pipeline__Road_traffic_density,cat_pipeline__Festival
0,-1.680369,-2.055867,-1.320419,-1.191266,1.106912,-0.139653
1,1.130443,0.523320,0.441078,-1.459228,1.106912,-0.139653
2,0.603416,0.200921,3.964072,1.195349,-1.298996,7.160600
3,0.427740,1.168117,0.441078,-0.641809,0.304942,-0.139653
4,1.657471,-0.121477,0.441078,-0.904513,1.106912,-0.139653
...,...,...,...,...,...,...
35646,0.954767,-1.088672,0.441078,0.734050,-1.298996,-0.139653
35647,-0.977666,0.845718,0.441078,-1.459305,-0.497027,-0.139653
35648,-1.329018,0.845718,-1.320419,-0.658158,-0.497027,-0.139653
35649,-0.274963,1.168117,-1.320419,-0.921855,1.106912,-0.139653


In [384]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [385]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [386]:
regression.score(X_test,y_test)

0.4837506658506936

In [387]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__multiple_deliveries,num_pipeline__Distance_Resturant_to_Location,cat_pipeline__Road_traffic_density,cat_pipeline__Festival
0,-1.680369,-2.055867,-1.320419,-1.191266,1.106912,-0.139653
1,1.130443,0.523320,0.441078,-1.459228,1.106912,-0.139653
2,0.603416,0.200921,3.964072,1.195349,-1.298996,7.160600
3,0.427740,1.168117,0.441078,-0.641809,0.304942,-0.139653
4,1.657471,-0.121477,0.441078,-0.904513,1.106912,-0.139653
...,...,...,...,...,...,...
35646,0.954767,-1.088672,0.441078,0.734050,-1.298996,-0.139653
35647,-0.977666,0.845718,0.441078,-1.459305,-0.497027,-0.139653
35648,-1.329018,0.845718,-1.320419,-0.658158,-0.497027,-0.139653
35649,-0.274963,1.168117,-1.320419,-0.921855,1.106912,-0.139653


In [388]:
import numpy as np

In [389]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squre = r2_score(true, predicted)

    return mae, rmse, r2_squre


In [390]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet(),
}

train_model_list =[]
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_predict = model.predict(X_test)

    mae, rmse, r2_squre = evaluate_model(y_test, y_predict)

    print(list(models.keys())[i])

    model_list.append(list(models.keys())[i])

    print("model Training performance")
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("R2_Square: ", r2_squre)

    r2_list.append(r2_squre)

    print("="*35)
    print("\n")

LinearRegression
model Training performance
RMSE:  6.74423021010064
MAE:  5.320816725408179
R2_Square:  0.4837506658506936


Lasso
model Training performance
RMSE:  7.032975908629033
MAE:  5.572219859969871
R2_Square:  0.43859924607734135


Ridge
model Training performance
RMSE:  6.7442293557262
MAE:  5.320814701441175
R2_Square:  0.4837507966499677


ElasticNet
model Training performance
RMSE:  7.146986142678009
MAE:  5.678036808842764
R2_Square:  0.4202501938064873


